In [1]:
import pandas as pd

In [104]:
df2=pd.read_csv("Data Science Jobs preprocessed.csv")
df=pd.read_csv("Data Science Jobs MAYBE.csv")
df.drop("Unnamed: 0",axis=1,inplace=True)
df2.drop("Unnamed: 0",axis=1,inplace=True)
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,Machine Learning,SQL,Computer Science,Deep Learning,Big Data/Spark,Visualization Tool,Data Science,Experience,Grade Required,Company Age
0,Data Analytics,$38.00 - $45.00 Per Hour (Employer est.),Minimum three years experience in data science...,3.9,J & S Consulting,"Tucker, GA",51 to 200 Employees,--,Company - Private,Information Technology Support Services,...,0,1,0,0,0,0,1,3,0,-1
1,Data Scientist,$120K - $130K (Employer est.),Data science: 1 year (Preferred). Expertise in...,4.0,element technologies,Remote,51 to 200 Employees,2000,Company - Private,Information Technology Support Services,...,1,0,0,0,0,0,1,1,0,24
2,Data Scientist,$66.82 - $80.54 Per Hour (Employer est.),End-to-end data and model pipeline deployment ...,4.4,Boston Technology,"Malvern, PA",51 to 200 Employees,2004,Company - Private,Information Technology Support Services,...,1,0,0,0,1,0,0,-1,0,20
3,Data Engineer,$105K - $160K (Employer est.),Communicates technical concepts to non-technic...,3.9,Costco Wholesale,"Dallas, TX",10000+ Employees,1976,Company - Public,General Merchandise & Superstores,...,1,0,0,0,0,0,0,-1,0,48
4,Data Science Co-Op,$84K - $109K (Glassdoor est.),Work with source control tools like GIT to sav...,4.1,Hunter Engineering Company,"Bridgeton, MO",1001 to 5000 Employees,1946,Company - Private,Machinery Manufacturing,...,1,0,1,0,0,0,0,-1,1,78


In [105]:
df=df[["Average Salary","Rating","Size","Type of ownership","Industry","Sector","Revenue","Employeer Estimate","State","Company Age",'Analysis','Machine Learning', 'SQL', 'Computer Science', 'Deep Learning','Big Data/Spark', 'Visualization Tool', 'Data Science',"Seniority"]]
df.head()

,Average Salary,Rating,Size,Type of ownership,Industry,Sector,Revenue,Employeer Estimate,State,Company Age,Analysis,Machine Learning,SQL,Computer Science,Deep Learning,Big Data/Spark,Visualization Tool,Data Science,Seniority
0,83000.0,3.9,51 to 200 Employees,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),1,GA,-1,0,0,1,0,0,0,0,1,Not Applicable
1,125000.0,4.0,51 to 200 Employees,Company - Private,Information Technology Support Services,Information Technology,Unknown / Non-Applicable,1,Remote,24,0,1,0,0,0,0,0,1,Mid-Level
2,147360.0,4.4,51 to 200 Employees,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),1,AL,20,0,1,0,0,0,1,0,0,Mid-Level
3,132500.0,3.9,10000+ Employees,Company - Public,General Merchandise & Superstores,Retail & Wholesale,Unknown / Non-Applicable,1,AL,48,0,1,0,0,0,0,0,0,Mid-Level
4,96500.0,4.1,1001 to 5000 Employees,Company - Private,Machinery Manufacturing,Manufacturing,$500 million to $1 billion (USD),0,ID,78,0,1,0,1,0,0,0,0,Not Applicable


# Preprocess for modelling

In [106]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pandas as pd

scaler_features = StandardScaler()
scaler_target = StandardScaler()

df['Company Age'] = scaler_features.fit_transform(df['Company Age'].values.reshape(-1, 1))
df['Rating'] = scaler_features.fit_transform(df['Rating'].values.reshape(-1, 1))
df['Average Salary'] = scaler_target.fit_transform(df['Average Salary'].values.reshape(-1, 1))

# One-hot encode categorical variables
df = pd.get_dummies(df, dtype=int)

# Split into features (X) and target (y)
X = df.drop(columns=['Average Salary'])
y = df['Average Salary']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ridge Regression

In [107]:
from sklearn.linear_model import Ridge

# Train Ridge Regression
ridge_model = Ridge(alpha=1.0)  # Adjust alpha for regularization strength
ridge_model.fit(X_train, y_train)

# Predict on test set
y_pred = ridge_model.predict(X_test)

# Inverse-transform predictions for interpretation
y_test_original = scaler_target.inverse_transform(y_test.values.reshape(-1, 1))
y_pred_original = scaler_target.inverse_transform(y_pred.reshape(-1, 1))

# Evaluate
mse = mean_squared_error(y_test_original, y_pred_original)
print(f"Ridge Regression MSE: {mse}")


Ridge Regression MSE: 2082744134.0594542


# Lasso Regression

In [108]:
from sklearn.linear_model import Lasso

# Train Lasso Regression
lasso_model = Lasso(alpha=0.01)  # Adjust alpha for regularization strength
lasso_model.fit(X_train, y_train)

# Print selected features
selected_features = X.columns[(lasso_model.coef_ != 0)]
print("Selected Features:", selected_features)

# Evaluate Lasso Model
y_pred_lasso = lasso_model.predict(X_test)

# Inverse-transform predictions
y_pred_original_lasso = scaler_target.inverse_transform(y_pred_lasso.reshape(-1, 1))
mse_lasso = mean_squared_error(y_test_original, y_pred_original_lasso)
print(f"Lasso Regression MSE: {mse_lasso}")

Selected Features: Index(['Employeer Estimate', 'Company Age', 'Analysis', 'Machine Learning',
       'SQL', 'Computer Science', 'Deep Learning', 'Big Data/Spark',
       'Visualization Tool', 'Data Science', 'Size_51 to 200 Employees',
       'Type of ownership_Company - Private',
       'Type of ownership_Company - Public',
       'Type of ownership_Nonprofit Organization',
       'Industry_Health Care Services & Hospitals', 'Sector_Healthcare',
       'Sector_Information Technology',
       'Sector_Pharmaceutical & Biotechnology',
       'Sector_Transportation & Logistics', 'Revenue_$1 to $5 billion (USD)',
       'Revenue_$10+ billion (USD)', 'Revenue_Unknown / Non-Applicable',
       'State_AK', 'State_AL', 'State_CA', 'State_NE', 'State_United States',
       'State_WA', 'Seniority_Junior', 'Seniority_Manager/Leadership',
       'Seniority_Mid-Level', 'Seniority_Senior'],
      dtype='object')
Lasso Regression MSE: 2164890279.165968


# Lasso and Ridge Combined

In [109]:
# Subset dataset with Lasso-selected features
selected_features_lasso = [
    'Employeer Estimate', 'Company Age', 'Analysis', 'Machine Learning', 'SQL',
    'Computer Science', 'Deep Learning', 'Big Data/Spark', 'Visualization Tool',
    'Data Science', 'Size_51 to 200 Employees',
    'Type of ownership_Company - Private', 'Type of ownership_Company - Public',
    'Type of ownership_Nonprofit Organization',
    'Industry_Health Care Services & Hospitals', 'Sector_Healthcare',
    'Sector_Information Technology', 'Sector_Pharmaceutical & Biotechnology',
    'Sector_Transportation & Logistics', 'Revenue_$1 to $5 billion (USD)',
    'Revenue_$10+ billion (USD)', 'Revenue_Unknown / Non-Applicable',
    'State_AK', 'State_AL', 'State_CA', 'State_NE', 'State_United States',
    'State_WA', 'Seniority_Junior', 'Seniority_Manager/Leadership',
    'Seniority_Mid-Level', 'Seniority_Senior'
]
X_train_subset = X_train[selected_features_lasso]
X_test_subset = X_test[selected_features_lasso]

ridge_model_subset = Ridge(alpha=1.0)
ridge_model_subset.fit(X_train_subset, y_train)
y_pred_subset = ridge_model_subset.predict(X_test_subset)

mse_subset = mean_squared_error(
    scaler_target.inverse_transform(y_test.values.reshape(-1, 1)),
    scaler_target.inverse_transform(y_pred_subset.reshape(-1, 1))
)
print(f"Ridge with Lasso-selected Features MSE: {mse_subset}")


Ridge with Lasso-selected Features MSE: 2194872221.3769855


In [110]:
import numpy as np

# Ridge coefficients
coefficients = ridge_model.coef_
features = X.columns
for feature, coef in zip(features, coefficients):
    print(f"{feature}: {coef:.4f}")

Rating: -0.0689
Employeer Estimate: 0.3282
Company Age: -0.0946
Analysis: -0.0692
Machine Learning: 0.1897
SQL: -0.1722
Computer Science: 0.1457
Deep Learning: 0.0309
Big Data/Spark: 0.1161
Visualization Tool: -0.2590
Data Science: -0.0317
Size_-1: -0.2444
Size_1 to 50 Employees: -0.0145
Size_10000+ Employees: -0.1204
Size_1001 to 5000 Employees: 0.1105
Size_201 to 500 Employees: 0.2939
Size_5001 to 10000 Employees: 0.0712
Size_501 to 1000 Employees: 0.1482
Size_51 to 200 Employees: -0.2608
Size_Unknown: 0.0163
Type of ownership_-1: -0.2444
Type of ownership_College / University: -0.0230
Type of ownership_Company - Private: 0.1691
Type of ownership_Company - Public: 0.0414
Type of ownership_Government: -0.1729
Type of ownership_Hospital: 0.8271
Type of ownership_Nonprofit Organization: -0.5837
Type of ownership_Private Practice / Firm: -0.2780
Type of ownership_School / School District: 0.0928
Type of ownership_Self-employed: 0.0404
Type of ownership_Subsidiary or Business Segment: 0.1

# Ridge Cross validation

In [111]:
from sklearn.model_selection import GridSearchCV

# Ridge cross-validation for alpha
ridge_params = {'alpha': np.logspace(-3, 3, 7)}
ridge_cv = GridSearchCV(Ridge(), ridge_params, scoring='neg_mean_squared_error', cv=5)
ridge_cv.fit(X_train, y_train)

print("Best alpha for Ridge:", ridge_cv.best_params_)
print("Best CV MSE for Ridge:", -ridge_cv.best_score_)


Best alpha for Ridge: {'alpha': 10.0}
Best CV MSE for Ridge: 0.7085077279733618


In [112]:
from sklearn.metrics import mean_squared_error

# Compute RMSE
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
print(f"Ridge Regression RMSE: {rmse}")


Ridge Regression RMSE: 45637.09164768778


# Gradient Boost

In [113]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Gradient Boosting
gbr = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=4, random_state=42)
gbr.fit(X_train, y_train)

# Predictions and RMSE
y_pred_gbr = gbr.predict(X_test)
rmse_gbr = np.sqrt(mean_squared_error(y_test, y_pred_gbr))
print(f"Gradient Boosting RMSE: {rmse_gbr}")

Gradient Boosting RMSE: 0.9442989515920165


In [114]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
}

grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best Gradient Boosting RMSE:", np.sqrt(-grid_search.best_score_))


Best parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000}
Best Gradient Boosting RMSE: 0.856794544747574


# Random Forest

In [115]:
from sklearn.ensemble import RandomForestRegressor

# Random Forest
rf = RandomForestRegressor(n_estimators=500, max_depth=10, random_state=42)
rf.fit(X_train, y_train)

# Predictions and RMSE
y_pred_rf = rf.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print(f"Random Forest RMSE: {rmse_rf}")

Random Forest RMSE: 0.8873227588057296


In [116]:
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [10, 20, None],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
}

grid_search_rf = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search_rf.fit(X_train, y_train)

print("Best parameters:", grid_search_rf.best_params_)
print("Best Random Forest RMSE:", np.sqrt(-grid_search_rf.best_score_))

c:\Users\testr\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
c:\Users\testr\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
c:\Users\testr\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

Best parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 1000}
Best Random Forest RMSE: 0.8512617188857754


testing the random forest on original scale because of improved rmse

In [117]:
# Invert scaling for predictions
y_pred_rf_original = mms2.inverse_transform(rf.predict(X_test).reshape(-1, 1))
y_test_original = mms2.inverse_transform(y_test.values.reshape(-1, 1))

# Calculate RMSE on the original scale
from sklearn.metrics import mean_squared_error
import numpy as np

rmse_rf_original = np.sqrt(mean_squared_error(y_test_original, y_pred_rf_original))
print(f"Random Forest RMSE (original scale): {rmse_rf_original}")


Random Forest RMSE (original scale): 43204.42712339326


In [119]:
mean_salary = df["Average Salary"].mean()
print(f"Mean Salary: {mms2.inverse_transform(mean_salary.reshape(-1, 1))}")


Mean Salary: [[170096.59877771]]


Random Forest is already giving a solid performance with a 25% relative RMSE.

# Hyperparameter randomized Search in Random Forest 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [500, 1000, 1500],
    'max_depth': [10, 20, None],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
}

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    scoring='neg_mean_squared_error',
    n_iter=50,
    cv=5,
    random_state=42
)

random_search.fit(X_train, y_train)
print("Best parameters:", random_search.best_params_)


Best parameters: {'n_estimators': 1000, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': None}


In [124]:
y_pred_rs = random_search.predict(X_test)
mse= np.sqrt(mean_squared_error(y_test, y_pred_rs))
print(f"Random Forest RMSE (original scale): {mse}")

Random Forest RMSE (original scale): 0.8893060239010974


In [125]:
# Invert scaling for predictions
y_pred_rs_original = mms2.inverse_transform(random_search.predict(X_test).reshape(-1, 1))
y_test_original = mms2.inverse_transform(y_test.values.reshape(-1, 1))

# Calculate RMSE on the original scale
from sklearn.metrics import mean_squared_error
import numpy as np

rmse_rf_original = np.sqrt(mean_squared_error(y_test_original, y_pred_rs_original))
print(f"Random Forest RMSE (original scale): {rmse_rf_original}")

Random Forest RMSE (original scale): 43300.99382522621


# XGBoost

In [129]:
!pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

# Assuming df is your DataFrame and already processed as per the previous steps
# Scaling the target (Average Salary) if needed
mms_salary = StandardScaler()
df['Average Salary'] = mms_salary.fit_transform(df[['Average Salary']])

# Prepare the data
X = df.drop(columns=['Average Salary'], axis=1)  # Drop target column
y = df['Average Salary']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.01)

# Hyperparameter tuning (GridSearchCV)
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [500, 1000],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

# Perform GridSearch
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Best parameters and model performance
best_params = grid_search.best_params_
print("Best Parameters for XGBoost:", best_params)

# Train the best model
best_xgb_model = grid_search.best_estimator_

# Predict on test data
y_pred = best_xgb_model.predict(X_test)

# Inverse transform salary predictions (back to original scale)
y_pred_original = mms_salary.inverse_transform(y_pred.reshape(-1, 1))

# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"XGBoost RMSE: {rmse}")

# For predicted salary values (inverse transformation)
print("Predicted Salary Mean (original scale):", y_pred_original.mean())

Best Parameters for XGBoost: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.8}
XGBoost RMSE: 0.9164216578690574
Predicted Salary Mean (original scale): 0.08126695


# Old

In [96]:
df=pd.get_dummies(df,dtype=int)

In [97]:
#df=df[["Average Salary","Rating","Size","Type of ownership","Industry","Sector","Revenue","Employeer Estimate","State","Company Age",'Analysis','Machine Learning', 'SQL', 'Computer Science', 'Deep Learning','Big Data/Spark', 'Visualization Tool', 'Data Science',"Seniority"]]
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

# Prepare the data
X = df.drop(columns=['Average Salary'],axis=1)  # Drop non-numeric/irrelevant columns
y = df['Average Salary']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Forward Selection
linear_regressor = LinearRegression()
forward_selector = SequentialFeatureSelector(linear_regressor, n_features_to_select="auto", direction="forward", scoring="neg_mean_squared_error", cv=5)

forward_selector.fit(X_train, y_train)

# Selected Features
selected_features = X_train.columns[forward_selector.get_support()]
print("Selected features:", selected_features)

# Evaluate the model with selected features
X_train_selected = forward_selector.transform(X_train)
X_test_selected = forward_selector.transform(X_test)

model = LinearRegression().fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)

mse = mean_squared_error(y_test, y_pred)
print(f"Forward Selection MSE: {mse}")

Selected features: Index(['Employeer Estimate', 'Machine Learning', 'SQL', 'Computer Science',
       'Big Data/Spark', 'Visualization Tool', 'Size_10000+ Employees',
       'Size_5001 to 10000 Employees', 'Size_51 to 200 Employees',
       'Size_Unknown', 'Type of ownership_Company - Private',
       'Type of ownership_Government', 'Type of ownership_Hospital',
       'Type of ownership_Nonprofit Organization',
       'Type of ownership_School / School District',
       'Type of ownership_Self-employed', 'Type of ownership_Unknown',
       'Industry_--', 'Industry_-1', 'Industry_Accounting & Tax',
       'Industry_Airlines, Airports & Air Transportation',
       'Industry_Banking & Lending', 'Industry_Beauty & Wellness',
       'Industry_Biotech & Pharmaceuticals', 'Industry_Business Consulting',
       'Industry_Car & Truck Rental', 'Industry_Chemical Manufacturing',
       'Industry_Computer Hardware Development', 'Industry_Construction',
       'Industry_Electronics Manufacturing',

In [102]:
y_test_inv=mms2.inverse_transform(y_test.values.reshape(-1,1))
y_pred_inv=mms2.inverse_transform(y_pred.reshape(-1,1))

mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f"Forward Selection MSE: {mse}")

Forward Selection MSE: 1.9735538918399645e+29


In [ ]:
import numpy as np

# Add a constant for statsmodels
X_with_const = sm.add_constant(X)

# Fit the full model
model = sm.OLS(y, X_with_const).fit()

while True:
    # Get the p-values and remove the feature with the highest p-value > 0.05
    p_values = model.pvalues
    max_p_value = p_values.max()
    if max_p_value > 0.05:
        excluded_feature = p_values.idxmax()
        print(f"Dropping '{excluded_feature}' with p-value {max_p_value}")
        X_with_const = X_with_const.drop(columns=[excluded_feature])
        model = sm.OLS(y, X_with_const).fit()
    else:
        break

print("Selected features after backward elimination:")
print(X_with_const.columns)

# Evaluate the final model
X_train, X_test, y_train, y_test = train_test_split(X_with_const, y, test_size=0.2, random_state=42)

final_model = LinearRegression().fit(X_train, y_train)
y_pred = final_model.predict(X_test)

mse = r2_score(y_test, y_pred)
print(f"Backward Elimination r2_score: {mse}")
mse = mean_squared_error(y_test, y_pred)
print(f"Backward Elimination MSE: {mse}")

Dropping 'Type of ownership_Unknown' with p-value 0.9955721379382576
Dropping 'State_IL' with p-value 0.98849023058045
Dropping 'State_TX' with p-value 0.9837638107622427
Dropping 'Industry_Advertising & Public Relations' with p-value 0.9834607572894192
Dropping 'Industry_Grocery Stores' with p-value 0.9831497688265312
Dropping 'State_KY' with p-value 0.9485517281357971
Dropping 'Industry_Food & Beverage Manufacturing' with p-value 0.9469035085111404
Dropping 'Sector_Retail & Wholesale' with p-value 0.9382826638922037
Dropping 'Sector_Transportation & Logistics' with p-value 0.9414299092194311
Dropping 'Industry_Accounting & Tax' with p-value 0.9258525460989905
Dropping 'Industry_Investment & Asset Management' with p-value 0.9283726312771816
Dropping 'Type of ownership_Subsidiary or Business Segment' with p-value 0.9265050051858008
Dropping 'Industry_Shipping & Trucking' with p-value 0.9239391997090518
Dropping 'Industry_Broadcast Media' with p-value 0.9211505750467561
Dropping 'State_

In [ ]:
df=df[["Average Salary","Rating","Size","Type of ownership","Industry","Sector","Revenue","Employeer Estimate","State","Company Age",'Analysis','Machine Learning', 'SQL', 'Computer Science', 'Deep Learning','Big Data/Spark', 'Visualization Tool', 'Data Science',"Seniority"]]
df2=df[["Average Salary","Rating","Revenue","Employeer Estimate","State","Company Age",'Analysis','Machine Learning', 'SQL', 'Computer Science', 'Deep Learning','Big Data/Spark', 'Visualization Tool', 'Data Science',"Seniority"]]

In [48]:
df2.columns

Index(['Average Salary', 'Employeer Estimate', 'State', 'Company Age',
       'Analysis', 'Machine Learning', 'SQL', 'Computer Science',
       'Deep Learning', 'Big Data/Spark', 'Visualization Tool', 'Data Science',
       'Seniority'],
      dtype='object')

In [69]:
from sklearn.preprocessing import StandardScaler
mms=StandardScaler()
mms2=StandardScaler()
df2['Company Age'] = mms.fit_transform(df2['Company Age'].values.reshape(-1,1))
df2["Average Salary"] = mms2.fit_transform(df2["Average Salary"].values.reshape(-1,1))

C:\Users\testr\AppData\Local\Temp\ipykernel_11016\4183182224.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Company Age'] = mms.fit_transform(df2['Company Age'].values.reshape(-1,1))
C:\Users\testr\AppData\Local\Temp\ipykernel_11016\4183182224.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Average Salary"] = mms2.fit_transform(df2["Average Salary"].values.reshape(-1,1))


In [70]:
df2=pd.get_dummies(df2,dtype=int)

In [71]:
from sklearn.model_selection import train_test_split
X=df2.drop("Average Salary",axis=1)
y=df2["Average Salary"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [72]:
from sklearn.ensemble import RandomForestRegressor
RFR=RandomForestRegressor()
model=RFR.fit(X_train,y_train)


In [73]:
y_pred=model.predict(X_test)

In [74]:
import numpy as np
from sklearn.metrics import r2_score
# y_test_inv=mms2.inverse_transform(y_test.values.reshape(-1,1))
# y_pred_inv=mms2.inverse_transform(y_pred.reshape(-1,1))
# from sklearn.metrics import mean_squared_error
# accuracy = mean_squared_error(y_test_inv,y_pred_inv)

inv_y_true = mms2.inverse_transform(y_test.values.reshape(-1,1))
inv_y_pred = mms2.inverse_transform(y_pred.reshape(-1,1))
itmse = r2_score(inv_y_true, inv_y_pred)
print("r2_score:", itmse)

r2_score: 0.2640802069288636


# ANN

In [42]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

model = Sequential()
model.add(Dense(256, input_shape=(X_train.shape[1],)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(128))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[reduce_lr])

Epoch 1/200


c:\Users\testr\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\testr\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 4.3923 - mae: 1.7086 - val_loss: 1.2260 - val_mae: 0.8150 - learning_rate: 0.0010
Epoch 2/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2144 - mae: 1.2096 - val_loss: 1.1900 - val_mae: 0.8195 - learning_rate: 0.0010
Epoch 3/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4331 - mae: 0.9371 - val_loss: 1.1821 - val_mae: 0.8200 - learning_rate: 0.0010
Epoch 4/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1149 - mae: 0.8352 - val_loss: 1.1919 - val_mae: 0.8228 - learning_rate: 0.0010
Epoch 5/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1058 - mae: 0.8474 - val_loss: 1.1903 - val_mae: 0.8172 - learning_rate: 0.0010
Epoch 6/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9371 - mae: 0.7739 - val_loss: 1.1792 - val_mae: 0.8125 - learning_rate: 0.0010
Epoch 7/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9053 - mae: 0.7412 - val_loss: 1.1337 - val_mae: 0.7959 - learning_rate: 0.0010
Epoch 8/200
15/15 ━━━━━━

In [292]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 256)            │        43,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 263,173 (1.00 MB)

 Trainable params: 87,425 (341.50 KB)

 Non-trainable params: 896 (3.50 KB)

 Optimizer params: 174,852 (683.02 KB)

In [286]:
model.fit(X_train, y_train, epochs=100, batch_size=5, validation_split=0.2)

Epoch 1/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.8429 - mae: 0.7019 - val_loss: 1.0452 - val_mae: 0.7385
Epoch 2/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.6587 - mae: 0.6481 - val_loss: 1.1071 - val_mae: 0.7443
Epoch 3/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.5588 - mae: 0.5819 - val_loss: 1.0803 - val_mae: 0.7422
Epoch 4/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.5469 - mae: 0.5881 - val_loss: 0.9999 - val_mae: 0.7297
Epoch 5/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.4450 - mae: 0.4909 - val_loss: 1.0272 - val_mae: 0.7164
Epoch 6/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.4405 - mae: 0.5171 - val_loss: 1.0219 - val_mae: 0.7180
Epoch 7/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.3377 - mae: 0.4430 - val_loss: 1.0248 - val_mae: 0.7422
Epoch 8/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.3109 - mae: 0.4061 - val_loss: 1.0481 - val_mae: 0.7212
Epoch 9/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.276

In [44]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Inverse transform the scaled target values
y_test_inv = mms2.inverse_transform(y_test.values.reshape(-1, 1))
y_pred_inv = mms2.inverse_transform(y_pred)

# Calculate Mean Absolute Error on the original scale
from sklearn.metrics import r2_score

mae = r2_score(y_test_inv, y_pred_inv)
print(f"r2 score: {mae}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
r2 score: 0.19187493510767306
